In [1]:
import mediapipe as mp
import numpy as np
import cv2
import time

# Initialize MediaPipe and OpenCV
cap = cv2.VideoCapture(0)
name = input("Enter the name of the data: ")

holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

X = []
data_size = 0

# Set the recording duration (in seconds)
record_duration = 3
start_time = time.time()

while True:
    lst = []

    # Read the frame
    ret, frm = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    frm = cv2.flip(frm, 1)

    # Process the frame
    try:
        res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    except Exception as e:
        print(f"Error processing frame: {e}")
        continue

    # Collect left hand landmarks
    if res.left_hand_landmarks:
        for i in res.left_hand_landmarks.landmark:
            lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
    else:
        lst.extend([0.0] * 42)

    # Collect right hand landmarks
    if res.right_hand_landmarks:
        for i in res.right_hand_landmarks.landmark:
            lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
    else:
        lst.extend([0.0] * 42)

    X.append(lst)
    data_size += 1

    # Draw the landmarks on the frame
    drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
    drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)

    # Display the frame and the number of captured frames
    cv2.putText(frm, str(data_size), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("window", frm)

    # Check if the recording duration has been reached or if 'Esc' key is pressed
    if cv2.waitKey(1) == 27 or (time.time() - start_time) > record_duration:
        break

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()

# Save the captured data
np.save(f"{name}.npy", np.array(X))
print(f"Data saved with shape: {np.array(X).shape}")

# Display output based on the recorded data
# (Insert additional code here if needed for specific output processing)



Enter the name of the data: happy
Data saved with shape: (30, 84)
